In [1]:
library(tidyverse)
library(data.table)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [2]:
# Load the cohort and the interval times -- interval times derived in 'create_action_space.ipynb'
data_path <- '../data/'
cohort_df <- read_csv(paste0(data_path, 'cohort.csv'))
interval_times_df <- read_csv(paste0(data_path, 'interval_times_df.csv'))

Parsed with column specification:
cols(
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  intime = col_datetime(format = ""),
  outtime = col_datetime(format = ""),
  suspected_infection_time_poe = col_datetime(format = ""),
  window_start = col_datetime(format = ""),
  window_end = col_datetime(format = ""),
  hospital_expire_flag = col_integer()
)
Parsed with column specification:
cols(
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  interval_start_time = col_datetime(format = ""),
  interval_end_time = col_datetime(format = "")
)


In [3]:
# Load the vitals
vitals_df <- read_csv(paste0(data_path, 'vitals_cohort.csv'))

Parsed with column specification:
cols(
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  window_start = col_datetime(format = ""),
  window_end = col_datetime(format = ""),
  intime = col_datetime(format = ""),
  outtime = col_datetime(format = ""),
  charttime = col_datetime(format = ""),
  vital_id = col_character(),
  valuenum = col_double()
)


In [4]:
head(vitals_df)

subject_id,hadm_id,icustay_id,window_start,window_end,intime,outtime,charttime,vital_id,valuenum
252,190159,221239,2133-03-30 15:30:00,2133-04-02 15:30:00,2133-03-31 04:24:30,2133-04-10 20:52:09,2133-03-31 04:38:00,Glucose,127
252,190159,221239,2133-03-30 15:30:00,2133-04-02 15:30:00,2133-03-31 04:24:30,2133-04-10 20:52:09,2133-03-31 05:00:00,DiasBP,77
252,190159,221239,2133-03-30 15:30:00,2133-04-02 15:30:00,2133-03-31 04:24:30,2133-04-10 20:52:09,2133-03-31 05:00:00,HeartRate,118
252,190159,221239,2133-03-30 15:30:00,2133-04-02 15:30:00,2133-03-31 04:24:30,2133-04-10 20:52:09,2133-03-31 05:00:00,MeanBP,96
252,190159,221239,2133-03-30 15:30:00,2133-04-02 15:30:00,2133-03-31 04:24:30,2133-04-10 20:52:09,2133-03-31 05:00:00,RespRate,12
252,190159,221239,2133-03-30 15:30:00,2133-04-02 15:30:00,2133-03-31 04:24:30,2133-04-10 20:52:09,2133-03-31 05:00:00,SpO2,100


In [5]:
# Cast to DT
vitals_dt <- as.data.table(vitals_df, key = 'icustay_id')
interval_times_dt <- as.data.table(interval_times_df, key = 'icustay_id')

In [6]:
merged_vitals <- interval_times_dt %>% 
                    merge(vitals_dt, allow.cartesian = TRUE)

In [18]:
mean_vitals <- merged_vitals[charttime > interval_start_time & charttime <= interval_end_time,
                                   .(mean_vital = mean(valuenum)), by = .(subject_id, 
                                                                          hadm_id, 
                                                                          icustay_id, 
                                                                          interval_start_time, 
                                                                          interval_end_time, 
                                                                          vital_id)]

In [8]:
head(mean_vitals)

subject_id,hadm_id,icustay_id,interval_start_time,interval_end_time,vital_id,mean_vital
4,185777,294638,2191-03-16,2191-03-16 04:00:00,HeartRate,94.75000
4,185777,294638,2191-03-16,2191-03-16 04:00:00,SpO2,98.75000
4,185777,294638,2191-03-16,2191-03-16 04:00:00,TempC,37.44445
4,185777,294638,2191-03-16,2191-03-16 04:00:00,DiasBP,60.00000
4,185777,294638,2191-03-16,2191-03-16 04:00:00,MeanBP,76.16670
4,185777,294638,2191-03-16,2191-03-16 04:00:00,SysBP,108.50000


In [9]:
labs_df <- read_csv(paste0(data_path, 'labs_cohort.csv'))

Parsed with column specification:
cols(
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  window_start = col_datetime(format = ""),
  window_end = col_datetime(format = ""),
  intime = col_datetime(format = ""),
  outtime = col_datetime(format = ""),
  charttime = col_datetime(format = ""),
  lab_id = col_character(),
  valuenum = col_double()
)


In [10]:
labs_dt <- as.data.table(labs_df, key = 'icustay_id')
merged_labs <- interval_times_dt %>% 
                    merge(labs_dt, allow.cartesian = TRUE)

In [17]:
mean_labs <- merged_labs[charttime > interval_start_time & charttime <= interval_end_time,
                                   .(mean_lab = mean(valuenum)), by = .(subject_id, 
                                                                          hadm_id, 
                                                                          icustay_id, 
                                                                          interval_start_time, 
                                                                          interval_end_time, 
                                                                          lab_id)]

In [19]:
mean_labs <- mean_labs %>% rename(meas_id = lab_id, mean_value = mean_lab)
mean_vitals <- mean_vitals %>% rename(meas_id = vital_id, mean_value = mean_vital)

mean_labs_vitals <- mean_labs %>% full_join(mean_vitals)

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time", "meas_id", "mean_value")


In [33]:
vitals_labs_spread <- mean_labs_vitals %>% 
                        spread(meas_id, mean_value) %>% 
                        right_join(interval_times_dt) 

# Exclude times in which no measurements were made
vitals_labs_spread_filt <- vitals_labs_spread %>%
                                gather(lab_id, meas_value, ALBUMIN:WBC) %>%
                                group_by(subject_id, hadm_id, icustay_id, interval_start_time, interval_end_time) %>%
                                summarise(exclude = all(is.na(meas_value))) %>%
                                full_join(vitals_labs_spread) %>%
                                filter(!exclude)

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")
Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")


In [34]:
head(vitals_labs_spread_filt)

subject_id,hadm_id,icustay_id,interval_start_time,interval_end_time,exclude,ALBUMIN,ANION GAP,BANDS,BICARBONATE,⋯,PLATELET,POTASSIUM,PT,PTT,RespRate,SODIUM,SpO2,SysBP,TempC,WBC
4,185777,294638,2191-03-15 12:00:00,2191-03-15 16:00:00,FALSE,NA,17,NA,24,⋯,207,3.1,12.3,31.3,NA,135,NA,NA,NA,9.7
4,185777,294638,2191-03-15 20:00:00,2191-03-16 00:00:00,FALSE,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,185777,294638,2191-03-16 00:00:00,2191-03-16 04:00:00,FALSE,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,98.75,108.5000,37.44445,NA
4,185777,294638,2191-03-16 04:00:00,2191-03-16 08:00:00,FALSE,2.8,15,NA,21,⋯,201,3.3,12.8,33.2,21.0,141,98.75,102.3333,36.66667,9.7
4,185777,294638,2191-03-16 08:00:00,2191-03-16 12:00:00,FALSE,NA,NA,NA,NA,⋯,NA,NA,NA,NA,24.0,NA,95.75,114.7500,37.11111,NA
4,185777,294638,2191-03-16 12:00:00,2191-03-16 16:00:00,FALSE,NA,NA,NA,NA,⋯,NA,NA,NA,NA,25.5,NA,97.00,126.5000,NA,NA


In [36]:
write_csv(vitals_labs_spread_filt, paste0(data_path, 'vitals_labs_spread_filt.csv'))